In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.patches as mplpatches
import plotnine as p9

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=200)

In [ ]:
D20_Lineage_assign=pd.read_csv(output_d+'/T5_control/AGG_Day20/outs/count/crispr_analysis/protospacer_calls_per_cell.csv',header=0,sep=',')

In [ ]:
D20_Lineage_assign

In [ ]:
adata = sc.read_10x_mtx(output_d+'/T5_control/AGG_Day20/outs/count/filtered_feature_bc_matrix',  
    var_names='gene_symbols',                  
    cache=True,gex_only=False)                                # write a cache file for faster subsequent reading

In [ ]:
adata.obs['cell_bc']=adata.obs.index

In [ ]:
adata.obs=adata.obs.reset_index().merge(D20_Lineage_assign,how="left",left_on=["cell_bc"],right_on=["cell_barcode"]).set_index('index')

In [ ]:
adata.obs.dtypes

In [ ]:
adata.obs['num_umis']=adata.obs['num_umis'].astype(str)

In [ ]:
adata.write(output_d+'/T5_control/Anndata/AggD20_all.h5ad')

In [ ]:
GEX_adata=adata[adata.obs['num_features']==1.0,adata.var["feature_types"] == "Gene Expression"]

In [ ]:
sc.pp.calculate_qc_metrics(GEX_adata, inplace=True)

In [ ]:
ribo_genes = GEX_adata.var_names.str.startswith(('RPS','RPL'))
GEX_adata.obs['percent_ribo'] = np.sum(
    GEX_adata[:, ribo_genes].X, axis=1).A1 / np.sum(GEX_adata.X, axis=1).A1

In [ ]:
mito_genes = GEX_adata.var_names.str.startswith(('MT-'))
GEX_adata.obs['percent_mito'] = np.sum(
    GEX_adata[:, mito_genes].X, axis=1).A1 / np.sum(GEX_adata.X, axis=1).A1

In [ ]:
sc.pp.normalize_total(GEX_adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(GEX_adata)

In [ ]:
GEX_adata.raw = GEX_adata


In [ ]:
GEX_adata.write(output_d+'/T5_control/Anndata/AggD20_GEX_Lineage_raw.h5ad')